<a href="https://colab.research.google.com/github/pierfrancescomartinello/NLP-Project/blob/main/rag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG pipelines applied to Unipa's website

## Dependencies
- `haystack-ai` is the preview of Haystack 2.0
- `sentence_transformers` is needed for embeddings
- `transformers` is needed to use open-source LLMs
- `accelerate` and `bitsandbytes` are required to use quantized versions of these models (with smaller memory footprint)
- `ragas-haystack` is required in order to define some metric of goodness for the models

In [1]:
%%capture
! pip install haystack-ai transformers accelerate bitsandbytes sentence_transformers ragas-haystack

<font color="red">WARNING: If on Google Colab, a session restart is to be done at this point in order to set all the new installed packages correctly</font>

Also a HuggingFace api key is necessary. It can be stored as a Secret with the name HF_TOKEN



## Setup

In [2]:
from IPython.display import Image
from pprint import pprint
import torch
import rich
import random

In [3]:
!wget https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json

--2024-06-19 00:10:21--  https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7606379 (7.3M) [text/plain]
Saving to: ‘unipa_dataset.json.1’

unipa_dataset.json. 100%[===================>]   7.25M  --.-KB/s    in 0.08s   

2024-06-19 00:10:21 (89.6 MB/s) - ‘unipa_dataset.json.1’ saved [7606379/7606379]



In [4]:
import pandas as pd
import numpy as np

# The dataset is loaded from Google Colab
filepath = "/content/unipa_dataset.json"
df = pd.read_json(filepath)
df.columns = ["title", "addr", "text"]

df

,title,addr,text
0,Università degli Studi di Palermo,https://www.unipa.it/,
1,Organi di Governo | Università degli Studi di ...,https://www.unipa.it/ateneo/OrganiDiGovernoECo...,
2,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
3,Presentazione | Università degli Studi di Palermo,https://www.unipa.it/ateneo/presentazione/,
4,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
...,...,...,...
6333,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Ca...
6334,| Università degli Studi di Palermo,https://www.unipa.it/amministrazione/rettorato...,
6335,Settore Comunicazione e URP | Settore Comunica...,https://www.unipa.it/amministrazione/rettorato...,
6336,Regolamenti per aree tematiche di interesse | ...,https://www.unipa.it/servizi/prevenzionedellac...,REGOLAMENTI PERSONALE DOCENTE E RICERCATORE


## Preprocessing

In [5]:
import re

# It has been noticed that there are some entries that consist only of a date in a particular format
def remove_dates(s: str) -> str:
    regex = r"\d{1,2}-(gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic)-\d{4}"

    return re.sub(regex, "", s)

# Empty strings are replaced with the numpy NaN representation so that dropna can be used
def remove_empty_docs(df: str) -> str:
    df["text"] = df["text"].replace("", np.nan)
    df.dropna(subset=["text"], inplace=True)

    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["text"].apply(remove_dates)
    df["text"] = df["text"].apply(lambda x: x.strip())

    df = remove_empty_docs(df)


    # Wpecify minimum document length
    # Most documents below threshold tend to be garbage documents
    df = df[df["text"].str.len() >= 200]

    df.reset_index(inplace=True, drop=True)

    return df

In [6]:
df = preprocess(df)
df

,title,addr,text
0,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
1,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
2,Sostegno allo studio | Centro per l’innovazion...,https://www.unipa.it/strutture/cimdu/Sostegno-...,"Da 15 anni ItaStra, Scuola di Lingua Italian..."
3,PNRR | PNRR | Università degli Studi di Palermo,https://www.unipa.it/progetti/pnrr/,Il Piano Nazionale di Ripresa e Resilienza - P...
4,Corsi di preparazione alle prove di accesso A....,https://www.unipa.it/Corsi-di-preparazione-all...,Sono aperte le iscrizioni all’edizione inverna...
...,...,...,...
2774,U.O. Didattica e Internazionalizzazione - Desc...,https://www.unipa.it/strutture/cimdu/U.O.-Dida...,Il Responsabile dell’U.O. Didattica e Interna...
2775,Ricercatori neoassunti | Università degli Stud...,https://www.unipa.it/strutture/cimdu/ricercato...,Per restare sempre aggiornati è possiblie iscr...
2776,Declaratorie della U.O. | Centro per l’innovaz...,https://www.unipa.it/strutture/cimdu/Declarato...,La U.O. Didattica e Internazionalizzazione si...
2777,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Cal...


In [7]:
from haystack.dataclasses import Document

# Population of the Document dataclasses. Title and URLs are added as metadata
titles = list(df["title"].values)
texts = list(df["text"].values)
urls = list(df["addr"].values)

raw_docs = []
for title, text, url in zip(titles, texts, urls):
    raw_docs.append(Document(content=text, meta={"name": title or "", "url": url}))

## Indexing Pipeline

In [8]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersTextEmbedder,
    SentenceTransformersDocumentEmbedder,
)
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

In [9]:
# We use In memory Document Store, albeit a different kind, like FAISSDocumentStore should be used for heavier document stores
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

In [10]:
# Creation of the indexing pipeline

indexing = Pipeline() # Empty Pipeline
# Cleaning the document from regex, headers, footers and similar
indexing.add_component("cleaner", DocumentCleaner())

# We split the documents in two sentence long subdocument
indexing.add_component(
    "splitter", DocumentSplitter(split_by="sentence", split_length=2)
)

# We transform the sentences in embeddings
indexing.add_component(
    "doc_embedder",
    SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large", # This is the model used
        device=ComponentDevice.from_str("cuda:0"), # We use the GPU for this operation
        meta_fields_to_embed=["title"], # We add the
    ),
)

# This simply writes in the document store
indexing.add_component(
    "writer",
    DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE),
)

# Cleaner -> Splitter -> Document Embedder -> Writer
indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "doc_embedder")
indexing.connect("doc_embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> doc_embedder.documents (List[Document])
  - doc_embedder.documents -> writer.documents (List[Document])

In [13]:
# This command takes the raw documents and transform them in embeddings
indexing.run({"cleaner": {"documents": raw_docs}})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   3%|3         | 21.0M/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Let's inspect the total number of chunked Documents and examine a Document

### Information about the embeddings

In [12]:
print(f"We have a total of {len(document_store.filter_documents())} documents in the document store")
print(f"The size of the embeddings is: {len(document_store.filter_documents()[0].embedding)}")

We have a total of 0 documents in the document store


NameError: name 'documen' is not defined

In [ ]:
pprint(document_store.filter_documents()[0])

## RAG Pipeline

In [ ]:
from haystack.components.generators import HuggingFaceLocalGenerator

### `zephyr-7b-beta`

- To load and manage Open Source LLMs in Haystack 2.0, we can use the `HuggingFaceLocalGenerator`.

- The LLM we choose is [Zephyr 7B Beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a fine-tuned version of Mistral 7B V.01 that focuses on helpfulness and outperforms many larger models on the MT-Bench and AlpacaEval benchmarks; the model was fine-tuned by the Hugging Face team.

- Since we are using a free Colab instance (with limited resources), we load the model using **4-bit quantization** (passing the appropriate `huggingface_pipeline_kwargs` to our Generator).
For an introduction to Quantization in Hugging Face Transformers, you can read [this simple blog post](https://huggingface.co/blog/merve/quantization).



In [ ]:
# Due to the limited resources, a 4-bit quantization is done
zephyr_generator = HuggingFaceLocalGenerator(
    #Name of the model
    "HuggingFaceH4/zephyr-7b-beta",
    huggingface_pipeline_kwargs={
        "device_map": "auto",
        "model_kwargs": {
            "load_in_4bit": True,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_compute_dtype": torch.bfloat16,
        },
    },
    generation_kwargs={"max_new_tokens": 500},
)

In [ ]:
zephyr_generator.warm_up()

### `falcon-7b`

In [ ]:
# Due to the limited resources, a 4-bit quantization is done
falcon_generator = HuggingFaceLocalGenerator(
    # Name of the model
    "tiiuae/falcon-7b",
    huggingface_pipeline_kwargs={
        "device_map": "auto",
        "model_kwargs": {
            "load_in_4bit": True,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_compute_dtype": torch.bfloat16,
        },
    },
    generation_kwargs={"max_new_tokens": 500},
)

In [ ]:
falcon_generator.warm_up()

### `PromptBuilder`

 It's a component that renders a prompt from a template string using Jinja2 engine.

 Let's setup our prompt builder, with a format like the following (appropriate for Zephyr):

 `"<|system|>\nSYSTEM MESSAGE</s>\n<|user|>\nUSER MESSAGE</s>\n<|assistant|>\n"`

In [ ]:
from haystack.components.builders import PromptBuilder

prompt_template = """<|system|>Using the information contained in the context, give a comprehensive answer to the question.
If the answer is contained in the context, also report the source URL.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
  Question: {{query}}
  </s>
<|assistant|>
"""
# Since an PromptBuilder object can be associated with only one Pipeline, two copies of the same object had to be made
zephyr_prompt_builder = PromptBuilder(template=prompt_template)
falcon_prompt_builder = PromptBuilder(template=prompt_template)

### Pipeline Creation

In [ ]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

Our RAG Pipeline finds Documents relevant to the user query and pass them to the LLM to generate a grounded answer.

It consists of several components:

- `SentenceTransformersTextEmbedder`: represent the query as a vector (capturing its meaning).
- `InMemoryEmbeddingRetriever`: finds the (top 5) Documents that are most similar to the query vector
- `PromptBuilder`
- `HuggingFaceLocalGenerator`

In [ ]:
zephyr_rag = Pipeline() # Empty Pipeline
# Text embedder
zephyr_rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large", # This is the model for the sentence transformer
        device=ComponentDevice.from_str("cuda:0") # We use the GPU
    ),
)
# Embedding Retriever
zephyr_rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(
        document_store=document_store,
        top_k=5
    )
)
zephyr_rag.add_component("prompt_builder", zephyr_prompt_builder)
zephyr_rag.add_component("llm", zephyr_generator)

# Text Embedder -> Retriever -> Prompt Builder -> LLM
zephyr_rag.connect("text_embedder", "retriever")
zephyr_rag.connect("retriever.documents", "prompt_builder.documents")
zephyr_rag.connect("prompt_builder.prompt", "llm.prompt")

In [ ]:
falcon_rag = Pipeline() # Empty Pipeline
# Text embedder
falcon_rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large",# This is the model for the sentence transformer
        device=ComponentDevice.from_str("cuda:0")# We use the GPU
    ),
)
# Embedding Retriever
falcon_rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(
        document_store=document_store,
        top_k=5
    )
)
falcon_rag.add_component("prompt_builder", falcon_prompt_builder)
falcon_rag.add_component("llm", falcon_generator)

# Text Embedder -> Retriever -> Prompt Builder -> LLM
falcon_rag.connect("text_embedder", "retriever")
falcon_rag.connect("retriever.documents", "prompt_builder.documents")
falcon_rag.connect("prompt_builder.prompt", "llm.prompt")

## Questions

In [ ]:
def get_generative_answer(query, model_object):

    # We pass the query to the particular model
    results = model_object.run(
        {"text_embedder": {"text": query}, "prompt_builder": {"query": query}}
    )
    # A reply is outputted
    answer = results["llm"]["replies"][0]
    rich.print(answer)


In [ ]:
def get_both_answer(query):
    print("ZEPHYR:")
    get_generative_answer(query, zephyr_rag)
    print("\nFALCON")
    get_generative_answer(query, falcon_rag)

### Zephyr

In [ ]:
while query:= input("Input your query (type EXIT to finish): ") != "EXIT":
    print(query)
    get_generative_answer(query, zephyr_rag)
    print("\n\n")

### Falcon

In [ ]:
while query:= input("Input your query (type EXIT to finish): ") != "EXIT":
    print(query)
    get_generative_answer(query, falcon_rag)
    print("\n\n")

### Both

In [ ]:
while query:= input("Input your query (type EXIT to finish): ") != "EXIT":
    print(query)
    get_both_answer(query)
    print("\n\n")

______

In [ ]:
from haystack import Pipeline

from haystack_integrations.components.evaluators.ragas import RagasEvaluator, RagasMetric

QUESTIONS = [
    "Which is the most popular global sport?",
    "Who created the Python language?",
]
CONTEXTS = [
    [
        "The popularity of sports can be measured in various ways, including TV viewership, social media presence, number of participants, and economic impact. Football is undoubtedly the world's most popular sport with major events like the FIFA World Cup and sports personalities like Ronaldo and Messi, drawing a followership of more than 4 billion people."
    ],
    [
        "Python, created by Guido van Rossum in the late 1980s, is a high-level general-purpose programming language. Its design philosophy emphasizes code readability, and its language constructs aim to help programmers write clear, logical code for both small and large-scale software projects."
    ],
]
RESPONSES = [
    "Football is the most popular sport with around 4 billion followers worldwide",
    "Python language was created by Guido van Rossum.",
]

GROUND_TRUTHS = [
    "Football is the most popular sport",
    "Python language was created by Guido van Rossum.",
]

ragas_pipe = Pipeline()
evaluator_context = RagasEvaluator(
    metric=RagasMetric.CONTEXT_PRECISION,
)
evaluator_aspect = RagasEvaluator(
    metric=RagasMetric.ASPECT_CRITIQUE,
    metric_params={"name": "custom", "definition": "Is this answer problematic for children?", "strictness": 3},
)
ragas_pipe.add_component("evaluator_context", evaluator_context)
ragas_pipe.add_component("evaluator_aspect", evaluator_aspect)

# Each metric expects a specific set of parameters as input. Refer to the
# Ragas class' documentation for more details.
results = ragas_pipe.run(
    {
        "evaluator_context": {"questions": QUESTIONS, "contexts": CONTEXTS, "ground_truths": GROUND_TRUTHS},
        "evaluator_aspect": {"questions": QUESTIONS, "contexts": CONTEXTS, "responses": RESPONSES},
    }
)

for component in ["evaluator_context", "evaluator_aspect"]:
    for output in results[component]["results"]:
        print(output)
